## 1. Import libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

## 2. Import data

In [2]:
path = r'C:\Users\Niklas Winter\Layoffs Analyse'

In [3]:
# 2.Importing data set Layoffs
df = pd.read_excel(os.path.join(path, '02 Data', 'Original Data', 'lay_off_data_clean.xlsx'))


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2348 entries, 0 to 2347
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Company         2348 non-null   object        
 1   Location        2348 non-null   object        
 2   Industry        2348 non-null   object        
 3   Laid_Off_Count  1607 non-null   float64       
 4   Date            2347 non-null   datetime64[ns]
 5   Funds_Raised    2137 non-null   object        
 6   Stage           2348 non-null   object        
 7   Country         2348 non-null   object        
 8   Percentage      1563 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 165.2+ KB


## Exploring the dataframe

In [5]:
df.head()

,Company,Location,Industry,Laid_Off_Count,Date,Funds_Raised,Stage,Country,Percentage
0,GoTo Group,Jakarta,Transportation,600.0,2023-03-10,1300.0,Post-IPO,Indonesia,NaN
1,Xero,Wellington,Finance,800.0,2023-03-09,681.0,Post-IPO,New Zealand,0.15
2,Shopee,Jakarta,Food,200.0,2023-03-09,NaN,Unknown,Indonesia,NaN
3,Gopuff,Philadelphia,Food,100.0,2023-03-09,3400.0,Series H,United States,0.02
4,Wave Financial,Toronto,Finance,50.0,2023-03-09,79.0,Acquired,Canada,NaN


In [6]:
df.tail()

,Company,Location,Industry,Laid_Off_Count,Date,Funds_Raised,Stage,Country,Percentage
2343,HopSkipDrive,Los Angeles,Transportation,8.0,2020-03-13,45.0,Unknown,United States,0.10
2344,Panda Squad,SF Bay Area,Consumer,6.0,2020-03-13,1.0,Seed,United States,0.75
2345,Tamara Mellon,Los Angeles,Retail,20.0,2020-03-12,90.0,Series C,United States,0.40
2346,EasyPost,Salt Lake City,Logistics,75.0,2020-03-11,12.0,Series A,United States,NaN
2347,Blackbaud,Charleston,Other,500.0,NaT,NaN,Post-IPO,United States,0.14


In [7]:
df.shape

(2348, 9)

# Data Check

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2348 entries, 0 to 2347
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Company         2348 non-null   object        
 1   Location        2348 non-null   object        
 2   Industry        2348 non-null   object        
 3   Laid_Off_Count  1607 non-null   float64       
 4   Date            2347 non-null   datetime64[ns]
 5   Funds_Raised    2137 non-null   object        
 6   Stage           2348 non-null   object        
 7   Country         2348 non-null   object        
 8   Percentage      1563 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(6)
memory usage: 165.2+ KB


### Observations: No Dtype change necessary, column names are logic

In [9]:
df.describe()

,Laid_Off_Count,Percentage
count,1607.000000,1563.000000
mean,238.470442,0.260401
std,772.151725,0.258458
min,3.000000,0.000000
25%,36.000000,0.100000
50%,80.000000,0.170000
75%,180.000000,0.300000
max,12000.000000,1.000000


### Observation: Max percentage of laid_off is 1, which means the company went bankrupt - worth to explore how often this happened!
### Big difference with min und max in column Laid_Off_Count, the mean seems a bit distorted due to that high max; The std of 772 supports that impression.

## # Data consistency Check


In [10]:
# Mixed-Type Data
for col in df.columns.tolist():
    weird = (df[[col]].applymap(type) !=df[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df[weird]) > 0:
        print (col)

Company
Date
Funds_Raised


### Observation: Columns Company, Date, Funds_Raised have mixed Data

In [11]:
# Fix column Company and date
df['Company'] = df['Company'].astype('str')
df['Date'] = df['Date'].astype('datetime64[ns]')


In [12]:
# Fixing column Funds_raised
df['Funds_Raised'] = df['Funds_Raised'].fillna(0)
df['Funds_Raised'] = df['Funds_Raised'].apply(lambda x: float(x) if isinstance(x, (int, float)) else 0)


In [13]:
# Missing Values
df.isnull().sum()

Company             0
Location            0
Industry            0
Laid_Off_Count    741
Date                1
Funds_Raised        0
Stage               0
Country             0
Percentage        785
dtype: int64

In [14]:
# Replace missing Laid_Off-Count with Median
df['Laid_Off_Count'].fillna(80, inplace = True)

In [15]:
# Drop missing Date value
df.dropna(subset = ['Date'], inplace = True)

In [16]:
 # Check amount of values over mean
count = (df['Percentage'] > 0.26041).value_counts()[True]
count

465

In [17]:
# Missing Values will be replaced by median
df['Percentage'].fillna(0.18, inplace = True)

In [18]:
# Missing Values check
df.isnull().sum() 

Company           0
Location          0
Industry          0
Laid_Off_Count    0
Date              0
Funds_Raised      0
Stage             0
Country           0
Percentage        0
dtype: int64

In [19]:
# Duplicates Check
df_dups = df[df.duplicated()]
df_dups

,Company,Location,Industry,Laid_Off_Count,Date,Funds_Raised,Stage,Country,Percentage
1483,Cazoo,London,Transportation,750.0,2022-06-07,0.0,Post-IPO,United Kingdom,0.15


In [20]:
df = df.drop_duplicates()

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2346 entries, 0 to 2346
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Company         2346 non-null   object        
 1   Location        2346 non-null   object        
 2   Industry        2346 non-null   object        
 3   Laid_Off_Count  2346 non-null   float64       
 4   Date            2346 non-null   datetime64[ns]
 5   Funds_Raised    2346 non-null   float64       
 6   Stage           2346 non-null   object        
 7   Country         2346 non-null   object        
 8   Percentage      2346 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(5)
memory usage: 183.3+ KB


In [23]:
#Export clean data
df.to_csv(os.path.join(path, '02 Data','Prepared Data', 'layoffs_clean.csv'))